In [1]:
import pandas as pd
import numpy as np

### Etapas:
- Escolher uma métrica para classificar 
- Decidir pré-requisitos para mostrar filmes
- Calcular score para cada filme que satisfaz condições
- Retornar lista final 

<img src="../../imgs/wr.png">

In [2]:
df = pd.read_csv('../../data/movies_metadata.csv',low_memory=False)
df.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [5]:
df.shape

(45466, 24)

In [10]:
#Calculata o número de votos acumulado
m = df['vote_count'].quantile(0.90)
m

160.0

In [11]:
#Filtra filmes maiores que 45 minutos e menores do que 300
q_movies = df[(df['runtime'] >= 45) & (df['runtime'] <= 300)]

#Considera filmes que possuem mais do que m votos
q_movies = q_movies[q_movies['vote_count'] >= m]

#Número de filmes final
q_movies.shape

(4512, 24)

In [12]:
# Calcula C
C = df['vote_average'].mean()
C

5.618207215133889

In [13]:
# Computa a classificação IMDB para cada filme
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    # Compute the weighted score
    return (v/(v+m) * R) + (m/(m+v) * C)

In [14]:
# Computa o score
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

In [15]:
#Ordena 
q_movies = q_movies.sort_values('score', ascending=False)

#Retorna os top 25 filmes
q_movies[['title', 'vote_count', 'vote_average', 'score', 'runtime']].head(25)

,title,vote_count,vote_average,score,runtime
314,The Shawshank Redemption,8358.0,8.5,8.445869,142.0
834,The Godfather,6024.0,8.5,8.425439,175.0
10309,Dilwale Dulhania Le Jayenge,661.0,9.1,8.421453,190.0
12481,The Dark Knight,12269.0,8.3,8.265477,152.0
2843,Fight Club,9678.0,8.3,8.256385,139.0
292,Pulp Fiction,8670.0,8.3,8.251406,154.0
522,Schindler's List,4436.0,8.3,8.206639,195.0
23673,Whiplash,4376.0,8.3,8.205404,105.0
5481,Spirited Away,3968.0,8.3,8.196055,125.0
2211,Life Is Beautiful,3643.0,8.3,8.187171,116.0
